In [1]:
# Importação de bibliotecas
import pandas as pd
import numpy as np

In [2]:
raiz = '/home/eduardo/repositorios/IS_Agro/databases'

### Ouro

##### Lavouras Temporárias

In [3]:
# Tabela de área colhida (ha) de lavouras temporárias
df_temp = (
    pd.read_parquet(f'{raiz}/prata_sidra_tab1612_lavoura_temporaria_area_colhida.parquet')
    .query('data >= 2000')
    .reset_index()
    .assign(geocodigo_uf = lambda df: df.geocodigo// 100000)
    .groupby(['geocodigo_uf', 'data', 'cultura'])
    .agg({'area':sum})
    .rename_axis(index={'geocodigo_uf': 'geocodigo'})
    .convert_dtypes()
)

# Tabela de área colhida (ha) da primeira safra de lavouras temporárias
df_ps = (
    pd.read_parquet(f'{raiz}/ouro_volume_agricola_lavouras_primeira_safra.parquet')
    .assign(geocodigo_uf = lambda df: df.geocodigo// 100000)
    .groupby(['geocodigo_uf', 'data', 'cultura'])
    .agg({'valor':sum})
    .rename_axis(index={'geocodigo_uf': 'geocodigo'})
    .convert_dtypes()
)

# Tabela de área colhida (ha) da segunda safra de lavouras temporárias
df_ss = (
    pd.read_parquet(f'{raiz}/ouro_volume_agricola_lavouras_segunda_safra.parquet')
    .assign(geocodigo_uf = lambda df: df.geocodigo// 100000)
    .groupby(['geocodigo_uf', 'data', 'cultura'])
    .agg({'valor':sum})
    .rename_axis(index={'geocodigo_uf': 'geocodigo'})
    .convert_dtypes()
)

# Tabela de área de uso agropecuário
df_agropecu = (
    pd.read_parquet(f'{raiz}/ouro_area_agropecuaria.parquet')
)

# Dados sobre área (por Mha) de Sistema Plantio Direto a nível estadual
df_spd = (
    pd.read_parquet(f'{raiz}/prata_SPD.parquet')
    .reset_index()
)

In [4]:
# Soja == 2713
(
    df_temp
    .query('cultura == 2713')
    .reset_index()
    .drop(columns='cultura')
    .merge(
        df_ss
        .query('cultura == 2711')
        .reset_index()
        .drop(columns='cultura'),
        how='left',
        on=['geocodigo', 'data']
    )
    .assign(
        area_SPD = lambda ha: ((ha.area / (ha.area + ha.valor)) * ha.area)
    )
    .assign(**{
        'Soja - Convencional - Monocultura': lambda ha: round(((ha.area - ha.area_SPD) / 1000000) * 0.5, 3), # 50% área soja - área soja SPD
        'Soja - Convencional - Sucessão': lambda ha: round(((ha.area - ha.area_SPD) / 1000000) * 0.5, 3), # 50% área soja - área soja SPD
        'Soja - Plantio Direto': lambda ha: round((ha.area_SPD / 1000000) * 0.7, 3), # 70% área soja SPD
        'Soja - SPD bem feito': lambda ha: round((ha.area_SPD / 1000000) * 0.3, 3) # 30% área soja SPD
    })
    .drop(columns=(['area', 'valor', 'area_SPD']))
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .dropna(subset='valor')
    .convert_dtypes()
    #.groupby(['data', 'cultura']).agg({'valor': sum})
)

,geocodigo,data,cultura,valor
0,11,2003-01-01,Soja - Convencional - Monocultura,0.015
1,11,2004-01-01,Soja - Convencional - Monocultura,0.019
2,11,2005-01-01,Soja - Convencional - Monocultura,0.023
3,11,2006-01-01,Soja - Convencional - Monocultura,0.028
4,11,2007-01-01,Soja - Convencional - Monocultura,0.026
...,...,...,...,...
1567,53,2018-01-01,Soja - SPD bem feito,0.015
1568,53,2019-01-01,Soja - SPD bem feito,0.017
1569,53,2020-01-01,Soja - SPD bem feito,0.017
1570,53,2021-01-01,Soja - SPD bem feito,0.02


In [39]:
# Milho == 2711
(
    df_ps
    .query('cultura == 2711')
    .reset_index()
    .assign(**{
        'Milho - Monocultura (1a. safra)': lambda ha: round((ha.valor / 1000000) * 0.2, 3), # (27% x 25%, 20% de área)
        'Milho (1a. safra)': lambda ha: round((ha.valor / 1000000) * 0.8, 3), # (27% x 75%, 80% de área)
        # 'Milho - 2a. safra (verão)': lambda x: round((x.area/1000000)*0.16, 3), # (73% x 20%, 16% de área)
        # 'Milho - 2a. safra + braquiária': lambda x: round((x.area/1000000)*0.3, 3), # (73% x 40%, 30% de área)
        # 'Milho - 2a. safra - SPD': lambda x: round((x.area/1000000)*0.3, 3) # (73% x 40%, 30% de área)
    })
    .drop(columns=(['cultura', 'valor']))
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .dropna(subset='valor')
    .convert_dtypes()
    #.groupby(['data', 'cultura']).agg({'valor': sum})
)

valor
data       cultura                               
2003-01-01 Milho (1a. safra)                7.788
           Milho - Monocultura (1a. safra)  1.948
2004-01-01 Milho (1a. safra)                7.529
           Milho - Monocultura (1a. safra)  1.881
2005-01-01 Milho (1a. safra)                6.866
           Milho - Monocultura (1a. safra)  1.714
2006-01-01 Milho (1a. safra)                7.426
           Milho - Monocultura (1a. safra)  1.858
2007-01-01 Milho (1a. safra)                7.821
           Milho - Monocultura (1a. safra)  1.957
2008-01-01 Milho (1a. safra)                 7.53
           Milho - Monocultura (1a. safra)  1.884
2009-01-01 Milho (1a. safra)                7.567
           Milho - Monocultura (1a. safra)  1.895
2010-01-01 Milho (1a. safra)                6.109
           Milho - Monocultura (1a. safra)  1.526
2011-01-01 Milho (1a. safra)                6.007
           Milho - Monocultura (1a. safra)  1.499
2012-01-01 Milho (1a. safra)                5.516
           Milho - Monocultura (1a. safra)  1.381
2013-01-01 Milho (1a. safra)                5.036
           Milho - Monocultura (1a. safra)   1.26
2014-01-01 Milho (1a. safra)                4.857
           Milho - Monocultura (1a. safra)  1.212
2015-01-01 Milho (1a. safra)                4.497
           Milho - Monocultura (1a. safra)  1.123
2016-01-01 Milho (1a. safra)                3.956
           Milho - Monocultura (1a. safra)  0.989
2017-01-01 Milho (1a. safra)                4.231
           Milho - Monocultura (1a. safra)  1.057
2018-01-01 Milho (1a. safra)                3.847
           Milho - Monocultura (1a. safra)  0.962
2019-01-01 Milho (1a. safra)                3.765
           Milho - Monocultura (1a. safra)  0.944
2020-01-01 Milho (1a. safra)                3.879
           Milho - Monocultura (1a. safra)   0.97
2021-01-01 Milho (1a. safra)                3.874
           Milho - Monocultura (1a. safra)  0.967
2022-01-01 Milho (1a. safra)                4.037
           Milho - Monocultura (1a. safra)   1.01

In [56]:
# Trigo == 2716 (revisão)
(
    df_ss
    .query('cultura == 2716')
    .reset_index()
    .assign(**{
        'Trigo - Convencional (2a. safra)': lambda ha: round((ha.valor / 1000000) * 0.1, 3), # 10% de área
        'Trigo - Plantio Direto (no-till - 2a. safra)': lambda ha: round((ha.valor/ 1000000) * 0.5, 3), # 50% de área
        'Trigo - Cultivo mínimo, SPD bem feito (2a. safra)': lambda ha: round((ha.valor / 1000000) * 0.4, 3) # 40% de área
    })
    .drop(columns=(['cultura', 'valor']))
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .dropna(subset='valor')
    .convert_dtypes()
    #.groupby(['data', 'cultura']).agg({'valor': sum})
)

,geocodigo,data,cultura,valor
0,13,2022-01-01,Trigo - Convencional (2a. safra),0.0
1,23,2020-01-01,Trigo - Convencional (2a. safra),0.0
2,29,2003-01-01,Trigo - Convencional (2a. safra),0.0
3,29,2004-01-01,Trigo - Convencional (2a. safra),0.0
4,29,2005-01-01,Trigo - Convencional (2a. safra),0.0
...,...,...,...,...
541,53,2018-01-01,"Trigo - Cultivo mínimo, SPD bem feito (2a. safra)",0.0
542,53,2019-01-01,"Trigo - Cultivo mínimo, SPD bem feito (2a. safra)",0.0
543,53,2020-01-01,"Trigo - Cultivo mínimo, SPD bem feito (2a. safra)",0.0
544,53,2021-01-01,"Trigo - Cultivo mínimo, SPD bem feito (2a. safra)",0.0


In [68]:
# Algodão Herbáceo == 2689 (revisão)
(
    df_ps
    .query('cultura == 2698')
    .reset_index()
    .assign(**{
        'Algodão - Convencional - morro abaixo': lambda ha: round((ha.valor / 1000000) * 0.5, 4), # 40% de área
        'Algodão - Plano/ondulado, conservado': lambda ha: round((ha.valor / 1000000) * 0.5, 4) # 60% de área
    })
    .drop(columns=(['cultura', 'valor']))
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .dropna(subset='valor')
    .convert_dtypes()
    #.groupby(['data', 'cultura']).agg({'valor': sum})
)

,geocodigo,data,cultura,valor
0,41,2003-01-01,Algodão - Convencional - morro abaixo,0.0
1,41,2004-01-01,Algodão - Convencional - morro abaixo,0.0
2,41,2005-01-01,Algodão - Convencional - morro abaixo,0.0
3,41,2006-01-01,Algodão - Convencional - morro abaixo,0.0
4,41,2007-01-01,Algodão - Convencional - morro abaixo,0.0
...,...,...,...,...
115,50,2013-01-01,"Algodão - Plano/ondulado, conservado",0.0
116,50,2014-01-01,"Algodão - Plano/ondulado, conservado",0.0
117,50,2015-01-01,"Algodão - Plano/ondulado, conservado",0.0
118,50,2016-01-01,"Algodão - Plano/ondulado, conservado",0.0


In [63]:
# Cana-de-açúcar == 2696
(
    df_temp
    .query('cultura == 2696')
    .reset_index()
    .assign(**{
        'Cana - Cana-planta somente': lambda ha: round((ha.area / 1000000) * 0.2, 4), # 20% de área
        'Cana - Cana-planta + 4 cana-soca (5 colheitas)': lambda ha: round((ha.area / 1000000) * 0.8, 4) # 80% de área
    })
    .drop(columns=(['cultura', 'area']))
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .dropna(subset='valor')
    .convert_dtypes()
    #.groupby(['data', 'cultura']).agg({'valor': sum})
)


,geocodigo,data,cultura,valor
0,11,2003-01-01,Cana - Cana-planta somente,0.0001
1,11,2004-01-01,Cana - Cana-planta somente,0.0001
2,11,2005-01-01,Cana - Cana-planta somente,0.0001
3,11,2006-01-01,Cana - Cana-planta somente,0.0003
4,11,2007-01-01,Cana - Cana-planta somente,0.0002
...,...,...,...,...
1075,53,2018-01-01,Cana - Cana-planta + 4 cana-soca (5 colheitas),0.0002
1076,53,2019-01-01,Cana - Cana-planta + 4 cana-soca (5 colheitas),0.0002
1077,53,2020-01-01,Cana - Cana-planta + 4 cana-soca (5 colheitas),0.0002
1078,53,2021-01-01,Cana - Cana-planta + 4 cana-soca (5 colheitas),0.0002


In [61]:
# Mandioca == 2708
(
    df_temp
    .query('cultura == 2698')
    .reset_index()
    .assign(**{
        'Mandioca - Convencional - solo coberto (entressafra)': lambda ha: round(ha.area / 1000000, 4)
    })
    .drop(columns=(['cultura', 'area']))
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .dropna(subset='valor')
    .convert_dtypes()
    #.groupby(['data', 'cultura']).agg({'valor': sum})
)

,,valor
data,cultura,
2003-01-01,Mandioca - Convencional - solo coberto (entres...,0.0027
2004-01-01,Mandioca - Convencional - solo coberto (entres...,0.0034
2005-01-01,Mandioca - Convencional - solo coberto (entres...,0.0044
2006-01-01,Mandioca - Convencional - solo coberto (entres...,0.003
2007-01-01,Mandioca - Convencional - solo coberto (entres...,0.0039
2008-01-01,Mandioca - Convencional - solo coberto (entres...,0.0047
2009-01-01,Mandioca - Convencional - solo coberto (entres...,0.0036
2010-01-01,Mandioca - Convencional - solo coberto (entres...,0.0023
2011-01-01,Mandioca - Convencional - solo coberto (entres...,0.0023


In [9]:
# Arroz == 2692
(
    df_temp
    .query('cultura == 2692')
    .reset_index()
    .assign(**{
        'Arroz irrigado (SC + RS)': lambda ha: round((ha.area/1000000) * 0.8, 3)
    })
    .drop(columns=(['cultura', 'area']))
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .dropna(subset='valor')
    .convert_dtypes()
    #.groupby(['data', 'cultura']).agg({'valor': sum})
)

,,valor
data,cultura,
2003-01-01,Arroz irrigado (SC + RS),2.545
2004-01-01,Arroz irrigado (SC + RS),2.986
2005-01-01,Arroz irrigado (SC + RS),3.131
2006-01-01,Arroz irrigado (SC + RS),2.379
2007-01-01,Arroz irrigado (SC + RS),2.311
2008-01-01,Arroz irrigado (SC + RS),2.281
2009-01-01,Arroz irrigado (SC + RS),2.294
2010-01-01,Arroz irrigado (SC + RS),2.177
2011-01-01,Arroz irrigado (SC + RS),2.199


In [113]:
# Cereais diversos:
# Aveia == 2693
# Centeio == 2698
# Cevada == 2699
# Feijão == 2702
# Girassol == 109179
# Mamona == 2707
# Sorgo == 2714
# Triticale == 109180
(
    df_temp
    .query('cultura in (2692, 2693, 2698, 2699, 2702, 109179, 2714, 2707, 109180)')
    .groupby(['geocodigo', 'data']).agg({'area': sum})
    .reset_index()
    .assign(**{
        'Aveia (cereais) - Convencional - Sem Cobertura': lambda x: round((x.area/1000000)*0.3, 4), # 30% de área
        'Aveia (cereais) - Convencional': lambda x: round((x.area/1000000)*0.4, 4), # 40% de área
        'Aveia (cereais) - SPD': lambda x: round((x.area/1000000)*0.3, 4) # 30% de área
    })
    #.groupby('data').agg({'Aveia (cereais) - Convencional - Sem Cobertura':sum})
    .drop(columns=('area'))
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .dropna(subset='valor')
    .convert_dtypes()
)

,geocodigo,data,cultura,valor
0,11,2003-01-01,Aveia (cereais) - Convencional - Sem Cobertura,0.0354
1,11,2004-01-01,Aveia (cereais) - Convencional - Sem Cobertura,0.0436
2,11,2005-01-01,Aveia (cereais) - Convencional - Sem Cobertura,0.0476
3,11,2006-01-01,Aveia (cereais) - Convencional - Sem Cobertura,0.0399
4,11,2007-01-01,Aveia (cereais) - Convencional - Sem Cobertura,0.0397
...,...,...,...,...
1615,53,2018-01-01,Aveia (cereais) - SPD,0.007
1616,53,2019-01-01,Aveia (cereais) - SPD,0.0057
1617,53,2020-01-01,Aveia (cereais) - SPD,0.0087
1618,53,2021-01-01,Aveia (cereais) - SPD,0.0058


##### Lavouras Permanentes

In [4]:
df_perm = (
    pd.read_parquet(f'{raiz}/prata_sidra_tab1613_lavoura_permanente_area_colhida.parquet')
    .query('data >= 2000')
    .reset_index()
    .assign(geocodigo_uf = lambda df: df.geocodigo // 100000)
    .groupby(['geocodigo_uf', 'data', 'cultura'])
    .agg({'area':sum})
    .rename_axis(index={'geocodigo_uf': 'geocodigo'})
    .convert_dtypes()
)

In [67]:
# Café == 2723
(
    df_perm
    .query('cultura == 2723')
    .reset_index()
    .assign(**{
        'Café - Convencional + cobertura do solo': lambda ha: round(ha.area / 1000000, 4) # Total das áreas de cultivo dos cafés em grão arábica e canephora
    })
    .drop(columns=(['cultura', 'area']))
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .dropna(subset='valor')
    .convert_dtypes()
    #.groupby(['data', 'cultura']).agg({'valor': sum})
)

,geocodigo,data,cultura,valor
0,11,2000-01-01,Café - Convencional + cobertura do solo,0.1989
1,11,2001-01-01,Café - Convencional + cobertura do solo,0.2229
2,11,2002-01-01,Café - Convencional + cobertura do solo,0.141
3,11,2003-01-01,Café - Convencional + cobertura do solo,0.1885
4,11,2004-01-01,Café - Convencional + cobertura do solo,0.1712
...,...,...,...,...
395,53,2018-01-01,Café - Convencional + cobertura do solo,0.0003
396,53,2019-01-01,Café - Convencional + cobertura do solo,0.0004
397,53,2020-01-01,Café - Convencional + cobertura do solo,0.0004
398,53,2021-01-01,Café - Convencional + cobertura do solo,0.0004


In [12]:
# Frutíferas:
# Abacate == 2717
# Abacaxi == 2688 - lav. temporária
# Açaí == 45981
# Banana == 2720
# Caqui == 2724
# Castanha de caju == 2725
# Figo == 2730
# Goiaba == 2731
# Laranja == 2733
# Limão == 2734
# Maçã == 2735
# Mamão == 2736
# Manga == 2737
# Maracujá == 2738
# Marmelo == 2739
# Melancia == 2709 - lav. temporária
# Melão == 2710 - lav. temporária
# Uva == 2748
# Pera == 2741
# Pêssego == 2742
# Tangerina == 2745
(
    pd.concat([
        df_perm
        .query('cultura in (2717, 45981, 2720, 2724, 2725, 2730, 2731, 2733, 2734, 2735, 2736, 2737, 2738, 2739, 2748, 2741, 2742, 2745)'),
        df_temp
        .query('cultura in (2688, 2709, 2710)')
    ])
    .groupby(['geocodigo', 'data']).agg({'area': sum})
    .reset_index()
    .assign(**{
         'Frutíferas - Sem cobertura do solo - Solo nu': lambda x: round((x.area/1000000) * 0.5, 2),
         #'Frutíferas - Com proteção à erosão': lambda x: round((x.area/1000000) * 0.5, 3)
    })
    .drop(columns=('area'))
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .dropna(subset='valor')
    .convert_dtypes()
    .groupby(['data']).agg({'valor':sum})
) # Quais devem ser consideradas para o total de perenes?

,valor
data,
2000-01-01,1.16
2001-01-01,1.15
2002-01-01,1.16
2003-01-01,1.24
2004-01-01,1.25
2005-01-01,1.25
2006-01-01,1.29
2007-01-01,1.32
2008-01-01,1.3


In [9]:

(
    pd.read_parquet(f'{raiz}/prata_sidra_tab1613_lavoura_permanente_area_destinada_colheita.parquet')
    .reset_index()
    .assign(geocodigo_uf = lambda df: df.geocodigo // 100000)
    .groupby(['geocodigo_uf', 'data', 'cultura'])
    .agg({'area':sum})
    .rename_axis(index={'geocodigo_uf': 'geocodigo'})
    .convert_dtypes()
    .query('cultura != 2723')
    .groupby(['geocodigo', 'data']).agg({'area': sum})
    .reset_index()
    .assign(**{
         'Frutíferas - Sem cobertura do solo - Solo nu': lambda x: round((x.area/1000000) * 0.5, 2),
         #'Frutíferas - Com proteção à erosão': lambda x: round((x.area/1000000) * 0.5, 3)
    })
    .drop(columns=('area'))
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .dropna(subset='valor')
    .convert_dtypes()
    .groupby(['data']).agg({'valor':sum})
) # Quais devem ser consideradas para o total de perenes?

,valor
data,
2003-01-01,1.97
2004-01-01,1.96
2005-01-01,1.99
2006-01-01,2.07
2007-01-01,2.08
2008-01-01,2.12
2009-01-01,2.04
2010-01-01,2.08
2011-01-01,2.07


In [11]:
(
    pd.concat([
        df_perm
        .query('cultura != 2723'),
        df_temp
        .query('cultura in (2688, 2709, 2710)')
    ])
    .groupby(['geocodigo', 'data']).agg({'area': sum})
    .reset_index()
    .assign(**{
         'Frutíferas - Sem cobertura do solo - Solo nu': lambda x: round((x.area/1000000) * 0.5, 1),
         #'Frutíferas - Com proteção à erosão': lambda x: round((x.area/1000000) * 0.5, 3)
    })
    .drop(columns=('area'))
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .dropna(subset='valor')
    .convert_dtypes()
    .groupby(['data']).agg({'valor':sum})
) 

,valor
data,
2000-01-01,1.0
2001-01-01,1.0
2002-01-01,1.1
2003-01-01,1.1
2004-01-01,1.1
2005-01-01,1.0
2006-01-01,1.0
2007-01-01,1.0
2008-01-01,1.0


##### Pastagens